In [1]:
import nltk , re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

In [2]:
url ='C:\\Users\\ASUS\\Documents\\txt_mining project\\files\\concat.html'
html = open(url, encoding='latin-1')
soup = BeautifulSoup(html,"lxml").get_text()

In [3]:
soup=soup.lstrip('\n').rstrip('\n')

In [4]:
para= soup.split(sep='\n\n')

In [5]:
# concatinate paragraphs in strings
para_tmp=[]
for s in para:
    para_tmp.append(s.replace('\n',' '))
para=para_tmp

In [6]:
#remove pages' numbers
para_tmp = []
for line in para:
    if len(line.lstrip(' ').rstrip(' ')) > 4:
        para_tmp.append(line)
para = para_tmp

In [7]:
para_compressed=[' ']
for line in para:
    if re.search('^.*[\\t]+.*$', line):
        para_compressed.append(line)
    else:
        para_compressed.append('\t\t' + line)

In [8]:
data='\n'.join(para_compressed)

In [9]:
import pandas as pd
from io import StringIO
import numpy as np
import string

In [10]:
TESTDATA = StringIO(data)

In [11]:
parser=pd.read_csv(TESTDATA,sep='\t',names=['is_header','section_type','section'])

In [12]:
#remove NaN rows
parser['is_header'] = parser['is_header'].fillna(False)
parser['section_type'] = parser['section_type'].fillna('None')
parser=parser.dropna()

In [13]:
#solving type problems
parser.is_header=parser.is_header.astype(bool)
parser.section=parser.section.astype(str)
parser.section_type=parser.section_type.astype(str)

In [14]:
#######  feature preparation  #######

In [15]:
def startWithArabic (Instr):
    """
    this function return true if the given string starts with am Arabic numeral
    """
    return Instr[:1].isdigit()
########################################
def startWithRoman (Instr):
    """
    this function return true if the given string starts with Simple Roman numeral followed by period or ' '
    """
    return re.match('^(?=[MDCLXVI])M*(C[MD]|D?C*)(X[CL]|L?X*)(I[XV]|V?I*)[ |.]',Instr[0:3]) is not None

#######################################
def leadingWhiteSpace(Instr):
    """
    this function return true if the given string starts with WhiteSpace character
    """
    return re.match('^\s',Instr) is not None

########################################
def ellipses(Instr):
    """
    this function return true if the given string contains ellipses
    """
    return re.search(r'(\w+)\.{3,}',Instr) is not None 
#######################################
def ContainsComma (Instr):
    return ',' in Instr
######################################
def stop_words_count(inputstr):
    var = inputstr
    return sum(1 for word in var.split() if word.lower() in stopwords.words('english') )
######################################
def num_Ponctuation(Instr):
    nonpuc=[c for c in Instr if c not in string.punctuation]
    nonpuc=''.join(nonpuc)
    if '.' in nonpuc:
        nonpuc = nonpuc + '.' # this line just to ignore the period for one time
    return len(Instr)-len(nonpuc)

In [16]:
#string.punctuation
parser.loc[:,'num_Ponctuation'] = parser['section'].apply(lambda st: num_Ponctuation(st)) #-->this is a bad metric

#stop_words_count
parser.loc[:,'stop_words_count'] = parser['section'].apply(lambda st: stop_words_count(st)) 

# LeadingAsterisk
#parser.loc[:,'LeadingAsterisk'] = parser['section'].apply(str.startswith, args='*') 

    # # leading arabic numeral
ArabicNumeral= parser['section'].str.lstrip(' ').apply(lambda st: startWithArabic(st))
    # # leading Roman numeral
RomanNumerals= parser['section'].str.lstrip(' ').apply(lambda st: startWithRoman(st))
# leadingNumeral 
parser.loc[:,'LeadingNumeral'] = ArabicNumeral | RomanNumerals

# endsInPeriod
parser.loc[:,'endsInPeriod'] = parser['section'].apply(str.endswith, args='.') 

# leadingWhiteSpace
#parser['leadingWhiteSpace'] = parser['section'].apply(lambda st: leadingWhiteSpace(st))

# ellipses
#parser.loc[:,'ellipses'] = parser['section'].apply(lambda st: ellipses(st))

#ContainsComma:
parser.loc[:,'ContainsComma'] = parser['section'].apply(lambda st: ContainsComma(st)) 

In [17]:
#remove spaces in begening and end of sections
parser.loc[:,'section'] = parser['section'].map(lambda x: x.lstrip(' ').rstrip(' ')) 
#length feature
parser.loc[:,'stringLength'] = parser['section'].apply(len)
#remove page's numbers
parser = parser.drop(parser[parser['stringLength']<4].index)

# percentCaps #presentation precision = 2
parser.loc[:,'percentCaps'] = parser['section'].apply(lambda st: round(sum(1 for c in st if c.isupper())*100/len(st),2))

In [18]:
parser

,is_header,section_type,section,num_Ponctuation,stop_words_count,LeadingNumeral,endsInPeriod,ContainsComma,stringLength,percentCaps
0,False,None,"Nieves v. Secretary, DOC",2,0,False,False,True,24,20.83
1,False,None,Description ORDER denying 1 Petition for wr...,8,10,False,False,False,225,6.67
2,False,None,IN THE UNITED STATES DISTRICT COURT ...,0,5,False,False,False,96,67.71
3,False,None,"SALSGIVER COMMUNICATIONS, INC., SALSGIVER TEL...",9,0,False,False,True,249,30.92
4,False,None,"CONSOLIDATED COMMUNICATIONS HOLDINGS, INC., N...",8,0,False,True,True,119,80.67
5,False,None,Defendants.,1,0,False,True,False,11,9.09
7,True,INTRODUCTION,I. Introduction and Background.,2,1,True,True,False,35,8.57
8,False,None,Before the Court are defendantsâ motion to d...,10,46,False,True,True,667,2.55
9,True,COMPLAINT,A. State Court Complaint.,2,0,False,True,False,25,16.00
10,False,None,"Plaintiffs Salsgiver Communications, Inc., Sal...",71,118,False,True,True,2683,3.43


In [19]:
parser = parser.reset_index(drop=True)

In [20]:
###### machine_learning_preprocessing ######

In [21]:
import numpy as np
import matplotlib.pyplot as plt

In [22]:
X=parser.iloc[:,[8,9]].values #[4,5,7,8,9]
Y=parser.iloc[:,0].values

In [23]:
#splitting data set to training set and test set
from sklearn.cross_validation import train_test_split

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [24]:
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, test_size=0.25, random_state=0)

In [25]:
#feature scaling
from sklearn.preprocessing import StandardScaler

In [26]:
sc_X= StandardScaler()
X_train=sc_X.fit_transform(X_train)
X_test=sc_X.transform(X_test)

In [27]:
#fiting logistic regression to the training set
from sklearn.linear_model import LogisticRegression

In [28]:
classifier=LogisticRegression(random_state=0)
classifier.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [29]:
#predict the test set result
#Y_pred=classifier.predict(X_train)
Y_pred=classifier.predict(X)

In [30]:
#making confusion Matrix
from sklearn.metrics import confusion_matrix
#cm=confusion_matrix(Y_train,Y_pred)
cm=confusion_matrix(Y,Y_pred)
print(cm)

[[152   0]
 [ 49  11]]


In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(Y, Y_pred)

0.76886792452830188

In [32]:
from sklearn.metrics import recall_score
recall_score(Y, Y_pred)

0.18333333333333332

In [33]:
from sklearn.metrics import precision_score
precision_score(Y, Y_pred)

1.0

In [34]:
#analyse the error source

In [35]:
len(Y_pred)

212

In [36]:
a=np.logical_xor(Y_pred,Y)

In [37]:
###### part_2_preprocessing: ######

In [38]:
# pure second part (without error propagation)--> all headers are selected

In [39]:
Aheader = parser[['section_type','section']]
Aheader = Aheader.drop(Aheader[Aheader['section_type'] == 'None'].index)

In [40]:
Aheader

,section_type,section
6,INTRODUCTION,I. Introduction and Background.
8,COMPLAINT,A. State Court Complaint.
10,NOTICE,B.Notice of Removal.
12,MOTION,C. Motion to Remand.
14,JURISDICTIONAL STATEMENT,II. Removal Jurisdiction.
15,PRELIMINARY STATEMENT,A. In General.
20,REMOVAL,B.Federal Question Removal.
23,JURISDICTIONAL STATEMENT,C. âArising Underâ Jurisdiction.
27,PREEMPTION,D. âArtful Pleadingâ Exception - Complete ...
34,PREEMPTION,E.âOrdinaryâ Preemption as a Defense.


In [41]:
#extracting Features: pnoctuation -->stopwords --> stemming -->features(... matrix)

In [42]:
#pnoctuation
import string

In [44]:
Aheader.loc[:,'Non_Ponc'] = Aheader['section'].apply(lambda st: re.sub(r'[^a-zA-Z ]', '',st))

In [45]:
#stopwords

In [46]:
from nltk.corpus import stopwords

In [47]:
def clean (st):
    clean_st = [word for word in st.split() if (word.lower() not in stopwords.words('english')) & (len(word)>3)]
    return clean_st    

In [48]:
clean('2. Failing to Request A Limiting Instruction')

['Failing', 'Request', 'Limiting', 'Instruction']

In [49]:
Aheader.loc[:,'clean_section'] = Aheader['Non_Ponc'].apply(lambda st:clean(st))

In [95]:
Aheader

In [51]:
#Stemming

In [52]:
from nltk.stem.snowball import SnowballStemmer

In [53]:
stemmer = SnowballStemmer('english')

In [54]:
for word in ['Precedent', 'Federal', 'Communications', 'Acts']:
    print(stemmer.stem(word))

preced
feder
communic
act


In [159]:
Aheader.loc[:,'stemm'] = Aheader['clean_section'].apply(lambda st: [stemmer.stem(word) for word in st])

In [160]:
Aheader.sort_values(by='section_type').reset_index(drop = True)

,section_type,section,Non_Ponc,clean_section,stemm
0,in order to show action in conformity therew...,"(1) If offered for some other purpose, such as...",If offered for some other purpose such as pro...,"[offered, purpose, proof, motive, opportunity,...","[offer, purpos, proof, motiv, opportun, intent..."
1,ACT,C. Precedent Under Federal Communications Acts.,C Precedent Under Federal Communications Acts,"[Precedent, Federal, Communications, Acts]","[preced, feder, communic, act]"
2,ACT,C. Precedent Under Federal Communications Acts.,C Precedent Under Federal Communications Acts,"[Precedent, Federal, Communications, Acts]","[preced, feder, communic, act]"
3,ACT,C. Precedent Under Federal Communications Acts.,C Precedent Under Federal Communications Acts,"[Precedent, Federal, Communications, Acts]","[preced, feder, communic, act]"
4,ACT,B. The Pole Attachment Act.,B The Pole Attachment Act,"[Pole, Attachment]","[pole, attach]"
5,ACT,B.The Pole Attachment Act.,BThe Pole Attachment Act,"[BThe, Pole, Attachment]","[bthe, pole, attach]"
6,ANALYSIS,Analysis,Analysis,[Analysis],[analysi]
7,ANALYSIS,II. ANALYSIS,II ANALYSIS,[ANALYSIS],[analysi]
8,ANALYSIS,II. Analysis,II Analysis,[Analysis],[analysi]
9,ANALYSIS,II. ANALYSIS,II ANALYSIS,[ANALYSIS],[analysi]


In [57]:
ll = Aheader.loc[Aheader.section_type == 'TESTIMONY'].reset_index(drop=True)
ll

,section_type,section,Non_Ponc,clean_section,stemm
0,TESTIMONY,C. Gravesâs Testimony,C Gravess Testimony,"[Gravess, Testimony]","[gravess, testimoni]"
1,TESTIMONY,1. Suzie's Testimony,Suzies Testimony,"[Suzies, Testimony]","[suzi, testimoni]"
2,TESTIMONY,a. Patty's testimony,a Pattys testimony,"[Pattys, testimony]","[patti, testimoni]"
3,TESTIMONY,b. Sam's testimony of physical abuse,b Sams testimony of physical abuse,"[Sams, testimony, physical, abuse]","[sam, testimoni, physic, abus]"


In [85]:
def aggreg (row):
    flist = []
    for ls in row['stemm'] :
        flist = flist + ls
    return flist

In [514]:
conca = (Aheader.groupby('section_type').apply(lambda lis:aggreg(lis))).to_frame('vocab') #.reset_index()
conca

,vocab
section_type,
"in order to show action in conformity therewith. It may, however, be admissible:","[offer, purpos, proof, motiv, opportun, intent..."
ACT,"[bthe, pole, attach, preced, feder, communic, ..."
ANALYSIS,"[analysi, analysi, analysi, analysi, analysi]"
APPLICATION,"[applic, iiiapplic]"
ARGUMENT,"[close, argument]"
BACKGROUND,"[background, background]"
COMPLAINT,"[state, court, complaint]"
CONCLUSION,"[conclus, conclus, conclus, conclus]"
DEFINITION,[definit]


In [359]:
# conca.vocab['TESTIMONY'].remove(conca.vocab['TESTIMONY'][0])
# conca.vocab['TESTIMONY']

In [480]:
#sort element in the vocab listes
conca.loc[:,'vocab']=conca['vocab'].apply(lambda st:sorted(st))

In [518]:
from sklearn.feature_extraction.text import CountVectorizer
#Tfidf
from sklearn.feature_extraction.text import TfidfTransformer

def  useful_words(ls):
    """
    this function takes a list of strings and return a list with strings used more than ones
    """
    bow_transformer = CountVectorizer().fit(ls)
    csr_matrix = bow_transformer.transform([' '.join(ls)])
    tfidf_transfrom = TfidfTransformer().fit_transform(csr_matrix)
    """eliminate"""
    tmp_list = []
    Mc=tfidf_transfrom.tocoo()
    for i in Mc.col:
        if Mc.data[Mc.col ==i][0] == Mc.data.min():
            tmp_list.append(bow_transformer.get_feature_names()[i])
    return list(set(ls)-set(tmp_list))

In [520]:
conca.loc[:,'vocab']=conca['vocab'].apply(lambda st: useful_words(st))

In [521]:
conca

,vocab
section_type,
"in order to show action in conformity therewith. It may, however, be admissible:",[offer]
ACT,"[attach, pole, preced, act, feder, communic]"
ANALYSIS,[]
APPLICATION,[]
ARGUMENT,[]
BACKGROUND,[]
COMPLAINT,[]
CONCLUSION,[]
DEFINITION,[]


In [373]:
# #conca = conca.set_index('section_type')

In [507]:
# from sklearn.feature_extraction.text import CountVectorizer

In [506]:
# bow_transformer = CountVectorizer().fit(conca.vocab['TESTIMONY']) # it takes a list of element #.set_index('section_type')

In [496]:
# bow_transformer.vocabulary_

In [497]:
# len(conca.vocab['TESTIMONY'])

In [498]:
# csr_matrix = bow_transformer.transform([' '.join(conca.vocab['TESTIMONY'])])#.set_index('section_type')
# print(csr_matrix)
# # csr_matrix.shape # -->Shape of Sparse Matrix
# # csr_matrix.nnz #-->Amount of Non-Zero occurences

In [499]:
# #sparsity
# sparsity = (100.0 * csr_matrix.nnz / (csr_matrix.shape[0] * csr_matrix.shape[1]))
# print('sparsity: {}'.format(round(sparsity)))

In [500]:
# csr_matrix [0,6]

In [501]:
#Tfidf
# from sklearn.feature_extraction.text import TfidfTransformer

In [502]:
# tfidf_transfrom = TfidfTransformer().fit_transform(csr_matrix)
# print(tfidf_transfrom.tocoo())

In [503]:
# tfidf_transfrom.min()
# tfidf_transfrom[0,6]

In [504]:
# tmp_list = []
# Mc=tfidf_transfrom.tocoo()
# for i in Mc.col:
#     if Mc.data[Mc.col ==i][0] == Mc.data.min():
#         tmp_list.append(bow_transformer.get_feature_names()[i])
# tmp_list

In [505]:
# a = list(set(conca.vocab['TESTIMONY'])-set(tmp_list)) #.set_index('section_type')

In [495]:
conca.xs('TESTIMONY')['vocab'] = a

In [303]:
# second part (as successor of part 1) -->headers recognized by predictor are selected

In [292]:
true_indices = list(np.argwhere(Y_pred == True).flatten())
true_indices

[53, 77, 104, 127, 136, 141, 144, 180, 183, 190, 210]

In [306]:
Pheaders = parser.iloc[true_indices, 1:3] 
Pheaders = Pheaders.reset_index(drop=True)
Pheaders

,section_type,section
0,SYLLABUS,SYLLABUS BY THE COURT
1,JURISDICTIONAL STATEMENT,JURISDICTIONAL DEFECT
2,OPINION,OPINION
3,STATEMENT OF FACTS,II. STATEMENT OF FACTS
4,STANDARDS OF REVIEW,IV. STANDARDS OF REVIEW
5,BACKGROUND,I. BACKGROUND
6,ANALYSIS,II. ANALYSIS
7,CONCLUSION,III. CONCLUSION
8,BACKGROUND,I. BACKGROUND
9,ANALYSIS,II. ANALYSIS
